# Szyfrowanie w pełni holomorficzne - Laboratorium 01

## Pierścienie ilorazowe wielomianów

Obiektem matematycznym powiązanym z ciałami Galois i używanym w kryptografi homomorficznej jest pierścień ilorazowy wielomianów $\mathbb{Z}_p[X]/W(X)$, gdzie $W(X)$ jest danym, konkretnym wielomianem stopnia $n$ a $p$ konkretną liczbą (najczęściej pierwszą).

$\mathbb{Z}_p[X]$ oznacza tutaj pierścień wielomianów dowolnych stopni o współczynnikach będących liczbami z $\mathbb{Z}_p$. Żeby otrzymać reprezentację wielomianu z $\mathbb{Z}[X]$ (tzn. wielomianu o współczynnikach całkowitych) w $\mathbb{Z}_p[X]$ należy obliczyć reprezentację jego współczynników $\mod p$.

Pierścień ilorazowy $\mathbb{Z}_p[X]/W(X)$ to mówiąc prostym językiem pierścień reszt z dzielenia wielomianów z $\mathbb{Z}_p[X]$ przez wielomian $W(X)$, czyli reprezentacją danego wielomianu staje się jego reszta z dzielenia przez $W(X)$.

## Zadanie 1.

Zaimplementować w Pythonie pierścień $\mathbb{Z}_{17}[X]/(X^4+1)$ wraz z arytmetyką, tzn. działaniami dodawania (+), odejmowania (-), mnożenia (\*) oraz mnożenia przez `int` (\*).

In [124]:
import numpy as np

x4p1 = np.array([1, 0, 0, 0, 1])

class Z17:
    def __init__(self, poly: np.array):
        _, p = np.polydiv(poly, x4p1)
        self.poly = (p % 17)

    def __mul__(self, other):
        if isinstance(other, Z17):
            return Z17(np.polymul(self.poly, other.poly))
        return Z17(self.poly * other)

    __rmul__ = __mul__

    def __add__(self, other):
        return Z17(np.polyadd(self.poly, other.poly))

    __radd__ = __add__

    def __neg__(self):
        return -1 * self

    def __repr__(self):
        p =  [(f"{a}" if a != 1 else "") + f"x^{3 - i}" for i, a in enumerate(self.poly[:-1].astype(int)) if a != 0] 
        if self.poly[-1] != 0:
            p += [f"{int(self.poly[-1])}"]
        return " +".join(p)

In [125]:
w1 = Z17([7,0,0,14,0,0,0])
w2 = Z17([24,0,-5,-7,13])
w3 = Z17([23,-3,1,35,0,4])

for w in [w1,w2,w3]:
    print(w)

14x^3 +10x^2
12x^3 +10x^2 +6
x^3 +x^2 +11x^1 +7


Dane testowe:

$$w1=7x^6+14x^3$$
$$w2=24x^4-5x^2-7x+13$$
$$w3=23x^5-3x^4+x^3+35x^2+4$$

Reprezentacja w $\mathbb{Z}_{17}[X]/(X^4+1)$:

$$w1=14x^3 + 10x^2$$
$$w2=12x^2 + 10x + 6$$
$$w3=x^3 + x^2 + 11x + 7$$

Arytmetyka:

$$w1+w2=14x^3 + 5x^2 + 10x + 6$$
$$w1*w2=14x^3 + 9x^2 + 2x + 12$$
$$6*w3=6x^3 + 6x^2 + 15x + 8$$
$$w3*6=6x^3 + 6x^2 + 15x + 8$$





In [119]:
print(w1+w2)
print(w1*w2)
print(6*w3)
print(w3*6)

14x^3 +5x^2 +10x^1 +6
14x^3 +9x^2 +2x^1 +12
6x^3 +6x^2 +15x^1 +8
6x^3 +6x^2 +15x^1 +8


## Algorytm BGV (Brakerski, Gentry, Vaikuntanathan 2011)

Parametry kryptosystemu:
- $n$ - stopień wielomianu $X^n+1$
- $q$ - podstawa arytmetyki modularnej
- $t$ - podstawa arytmetyki modularnej plaintextu, $t<<q$
- $\chi$ - dyskretny rokład typu Gaussowskiego
- $R_q=\mathbb{Z}_{q}[X]/(X^n+1)$

W uproszczonym modelu kryptosystemu przyjmijmy $n=4$, $q=17$, $t=2$.

`SecretKeyGen(params) -> sk`

- losujemy wektor $s\in\{-1,0,1\}^n$ z *binomial distribution* (prawdopodobieństwo wylosowania 0 jest największe, a prawdopodobieństwa wylosowania -1 i 1 są sobie równe)
- klucz prywatny $sk=s$
    

`PubKeyGen(sk, params) -> (pk0, pk1)`

- losujemy losowy element $a\in R_q$
- losujemy niewielki (w sensie współczynników) błąd $e\in R_q$ z rozkładu $\chi$
- $pk_0=as+te$
- $pk_1=-a$
- klucz publiczny $pk=(pk_0,pk_1)$

`Encrypt(m, pk, params) -> (c0, c1)`

- losujemy niewielkie (w sensie współczynników) błędy $e_0,e_1\in R_q$ z rozkładu $\chi$
- losujemy wektor $u\in\{-1,0,1\}^n$ z *binomial distribution*
- $c_0=pk_0\cdot u+te_0+m$
- $c_1=pk_1\cdot u+te_1$
- szyfrogram $c=(c_0,c_1)$

`Decrypt(c, sk, params)`

- obliczamy $m=c_0+c_1s\mod q\mod t$
- zwracamy $m$ jako odszyfrowaną wiadomość

## Zadanie 2.

Zaimplementuj uproszczoną wersję algorytmu BGV. Sprawdź poprawność deszyfrowania dla losowo generowanych wiadomości $m$.

In [148]:
n = 4 
q = 17
t = 2

def SecretKeyGen(n):
    sk =  np.random.binomial(2,0.5,size=n)-1
    return sk

def PubKeyGen(sk):
    a = Z17(np.random.randint(q,size=n))
    e = Z17(np.random.normal(1.5,size=n).astype(int))
    
    pk0 = a*sk + t*e
    pk1 = -a
    return (pk0,pk1)

def Encrypt(m,pk):
    e0 = Z17(np.random.normal(1.5,size=n).astype(int))
    e1 = Z17(np.random.normal(1.5,size=n).astype(int))
    u = np.random.binomial(2,0.5,size=n)-1
    c0 = pk[0]*u + t*e0 + m
    c1 = pk[1]*u + t*e1
    return (c0,c1)

def Decrypt(c, sk, q, t):
    m = c[0] + c[1]*sk
    m %= q
    m %= t
    return m

In [149]:
sk = SecretKeyGen(n)
pk = PubKeyGen(sk)

for p in np.random.randint(2,size=(10,4)):
    poly = Z17(p)
    c = Encrypt(poly,pk)
    m = Decrypt(c,sk,q,t)
    print(m == poly)\

TypeError: unsupported operand type(s) for %=: 'Z17' and 'int'